# 多面体描画

In [1]:
import numpy as np
import cdd
from itertools import combinations
MEPS = 1.0e-6

# 多面体を定義する不等式の作成
np.random.seed(2)
n, d = 40, 3
A = np.random.randint(0,100,(n,d))
b = np.sqrt(np.dot(A**2,np.ones(d))).astype(np.int64)
m, n = np.shape(A)

In [2]:
print(A.shape)
print(b.shape)

(40, 3)
(40,)


In [3]:
# pycddlib用のフォーマットに合わせる．
eb = np.hstack((b, np.zeros(n))).reshape(-1,1)
eA = np.vstack((-A,np.identity(n)))
ar = np.hstack((eb,eA))
mat = cdd.Matrix(ar,number_type='fraction')

A matrix [b -A] で Ax >= b を表す  
最後の 単位行列の部分はxが0以上であることを表す

In [4]:
print(mat)

begin
 43 4 rational
 83 -40 -15 -72
 95 -22 -43 -82
 82 -75 -7 -34
 130 -49 -95 -75
 115 -85 -47 -63
 97 -31 -90 -20
 85 -37 -39 -67
 66 -4 -42 -51
 76 -38 -33 -58
 130 -67 -69 -88
 107 -68 -46 -70
 129 -95 -83 -31
 116 -66 -80 -52
 91 -76 -50 -4
 135 -90 -63 -79
 77 -49 -39 -46
 52 -8 -50 -15
 28 -8 -17 -22
 129 -73 -57 -90
 141 -62 -83 -96
 59 -43 -32 -26
 77 -8 -76 -10
 79 -40 -34 -60
 111 -9 -70 -86
 91 -70 -19 -56
 106 -82 -1 -68
 109 -40 -81 -61
 120 -70 -97 -18
 150 -84 -90 -87
 70 -22 -43 -52
 136 -74 -72 -90
 165 -99 -91 -96
 61 -16 -55 -21
 129 -43 -93 -80
 109 -40 -70 -74
 71 -37 -59 -17
 83 -15 -30 -77
 78 -26 -39 -63
 66 -20 -22 -59
 55 -49 -27 0
 0 1 0 0
 0 0 1 0
 0 0 0 1
end


In [5]:
# 端点列挙実行
poly = cdd.Polyhedron(mat)
ext = poly.get_generators()

# 座標を計算
vl = np.array([np.array(v[1:])/v[0] for v in ext if v[0] != 0])
# 小数点形式に変換
vlist = vl.astype(np.float64)

In [6]:
# Fractionは分数の意味
for v in ext[:5]:
    print(v)

(Fraction(3293, 1391), 1, Fraction(3070, 1391), 0)
(Fraction(18143, 1206), Fraction(7781, 1206), Fraction(8303, 603), 1)
(Fraction(176, 125), Fraction(257, 250), 1, 0)
(Fraction(12709, 6020), Fraction(8937, 6020), 1, Fraction(95, 86))
(Fraction(2555, 816), Fraction(3631, 1632), 1, Fraction(7493, 3808))


In [7]:
print(vl[:5])

[[Fraction(1391, 3293) Fraction(3070, 3293) Fraction(0, 1)]
 [Fraction(7781, 18143) Fraction(16606, 18143) Fraction(1206, 18143)]
 [Fraction(257, 352) Fraction(125, 176) Fraction(0, 1)]
 [Fraction(8937, 12709) Fraction(6020, 12709) Fraction(6650, 12709)]
 [Fraction(3631, 5110) Fraction(816, 2555) Fraction(22479, 35770)]]


In [8]:
# vlistが端点
print(vlist[:5])

[[0.42241118 0.9322806  0.        ]
 [0.42887064 0.91528413 0.06647192]
 [0.73011364 0.71022727 0.        ]
 [0.70320245 0.47368007 0.52325124]
 [0.71056751 0.31937378 0.62843165]]


In [9]:
# 付加情報(枝の情報)の計算
# 端点を不等式に入れて 等しい関係になった3つの不等式の番号、setに入れる
zerosets = [set([i for i in range(m+n)
                 if abs(eb[i]+np.dot(eA[i],v)) <= MEPS]) for v in vl]

# 積集合を計算
# 2平面が一致した点同士を繋げる
elist = [[i,j] for i,j in combinations(range(len(vl)),2)
       if len(zerosets[i].intersection(zerosets[j])) >= 2]

In [10]:
print(zerosets[0])
print(zerosets[1])
print(zerosets[0].intersection(zerosets[1]))

{42, 27, 5}
{35, 27, 5}
{27, 5}


In [11]:
# 組み合わせを計算
for i,j in combinations(range(5),2):
    print(i,j)

0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


In [12]:
# 可視化
from vpython import *
# vpython 初期設定
scene = canvas(width = 800,height = 600)
vmin = np.min(vlist)-0.5
length = np.max(vlist)-vmin+0.5
scene.up = vector(0,0,1)
scene.forward = vector(-1,-1,-1)
scene.center = vector(0,0,0)
scene.range = 0.9*length
scene.background = color.white
cb = color.black
cr = color.red

# x,y,z軸を描画
arrow(pos=vector(vmin,0,0),axis=vector(length,0,0),
      shaftwidth=0.002,headwidth=0.05,color=cb)
arrow(pos=vector(0,vmin,0),axis=vector(0,length,0),
      shaftwidth=0.002,headwidth=0.05,color=cb)
arrow(pos=vector(0,0,vmin),axis=vector(0,0,length),
      shaftwidth=0.002,headwidth=0.05,color=cb)

# 頂点と稜線を描画
vertices = [sphere(pos=vector(*v),
                radius=0.01,color=cb) for v in vlist]
edges = [curve(pos=[vector(*vlist[i]), vector(*vlist[j])],
               radius=0.007,color=cb) for [i,j] in elist]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>